Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Neural style transfer on video
Using modified code from `pytorch`'s neural style [example](https://pytorch.org/tutorials/advanced/neural_style_tutorial.html), we show how to setup a pipeline for doing style transfer on video. The pipeline has following steps:
1. Split a video into images
2. Run neural style on each image using one of the provided models (from `pytorch` pretrained models for this example).
3. Stitch the image back into a video.

## Prerequisites
Make sure you go through the configuration Notebook located at https://github.com/Azure/MachineLearningNotebooks first if you haven't. This sets you up with a working config file that has information on your workspace, subscription id, etc. 

## Initialize Workspace

Initialize a workspace object from persisted configuration.

In [ ]:
import os
from azureml.core import Workspace, Run, Experiment

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

scripts_folder = "scripts_folder"

if not os.path.isdir(scripts_folder):
    os.mkdir(scripts_folder)

In [ ]:
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.datastore import Datastore
from azureml.data.data_reference import DataReference
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.pipeline.steps import PythonScriptStep, MpiStep
from azureml.core.runconfig import CondaDependencies, RunConfiguration

# Create or use existing compute

In [ ]:
# AmlCompute
cpu_cluster_name = "cpucluster"
try:
    cpu_cluster = AmlCompute(ws, cpu_cluster_name)
    print("found existing cluster.")
except:
    print("creating new cluster")
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_v2",
                                                                    max_nodes = 1)

    # create the cluster
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, provisioning_config)
    cpu_cluster.wait_for_completion(show_output=True)
    
# AmlCompute
gpu_cluster_name = "gpucluster"
try:
    gpu_cluster = AmlCompute(ws, gpu_cluster_name)
    print("found existing cluster.")
except:
    print("creating new cluster")
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_NC6",
                                                                    max_nodes = 3)

    # create the cluster
    gpu_cluster = ComputeTarget.create(ws, gpu_cluster_name, provisioning_config)
    gpu_cluster.wait_for_completion(show_output=True)

# Python Scripts
We use an edited version of `neural_style_mpi.py` (original is [here](https://github.com/pytorch/examples/blob/master/fast_neural_style/neural_style/neural_style_mpi.py)). Scripts to split and stitch the video are thin wrappers to calls to `ffmpeg`. 

We install `ffmpeg` through conda dependencies.

In [ ]:
import shutil
shutil.copy("neural_style_mpi.py", scripts_folder)

In [ ]:
%%writefile $scripts_folder/process_video.py
import argparse
import glob
import os
import subprocess

parser = argparse.ArgumentParser(description="Process input video")
parser.add_argument('--input_video', required=True)
parser.add_argument('--output_audio', required=True)
parser.add_argument('--output_images', required=True)

args = parser.parse_args()

os.makedirs(args.output_audio, exist_ok=True)
os.makedirs(args.output_images, exist_ok=True)

subprocess.run("ffmpeg -i {} {}/video.aac"
              .format(args.input_video, args.output_audio),
               shell=True, check=True
              )

subprocess.run("ffmpeg -i {} {}/%05d_video.jpg -hide_banner"
              .format(args.input_video, args.output_images),
               shell=True, check=True
              )

In [ ]:
%%writefile $scripts_folder/stitch_video.py
import argparse
import os
import subprocess

parser = argparse.ArgumentParser(description="Process input video")
parser.add_argument('--images_dir', required=True)
parser.add_argument('--input_audio', required=True)
parser.add_argument('--output_dir', required=True)

args = parser.parse_args()

os.makedirs(args.output_dir, exist_ok=True)

subprocess.run("ffmpeg -framerate 30 -i {}/%05d_video.jpg -c:v libx264 -profile:v high -crf 20 -pix_fmt yuv420p "
               "-y {}/video_without_audio.mp4"
               .format(args.images_dir, args.output_dir),
               shell=True, check=True
              )

subprocess.run("ffmpeg -i {}/video_without_audio.mp4 -i {}/video.aac -map 0:0 -map 1:0 -vcodec "
               "copy -acodec copy -y {}/video_with_audio.mp4"
               .format(args.output_dir, args.input_audio, args.output_dir),
               shell=True, check=True
              )

In [ ]:
# datastore for input video
account_name = "happypathspublic"
video_ds = Datastore.register_azure_blob_container(ws, "videos", "videos",
                                            account_name=account_name, overwrite=True)

# datastore for models
models_ds = Datastore.register_azure_blob_container(ws, "models", "styletransfer", 
                                                        account_name="pipelinedata", 
                                                        overwrite=True)
                                                        
# downloaded models from https://pytorch.org/tutorials/advanced/neural_style_tutorial.html are kept here
models_dir = DataReference(data_reference_name="models", datastore=models_ds, 
                           path_on_datastore="saved_models", mode="download")

# the default blob store attached to a workspace
default_datastore = ws.get_default_datastore()

# Sample video

In [ ]:
orangutan_video = DataReference(datastore=video_ds,
                            data_reference_name="video",
                            path_on_datastore="orangutan.mp4", mode="download")

In [ ]:
cd = CondaDependencies()

cd.add_channel("conda-forge")
cd.add_conda_package("ffmpeg")

cd.add_channel("pytorch")
cd.add_conda_package("pytorch")
cd.add_conda_package("torchvision")

# Runconfig
amlcompute_run_config = RunConfiguration(conda_dependencies=cd)
amlcompute_run_config.environment.docker.enabled = True
amlcompute_run_config.environment.docker.gpu_support = True
amlcompute_run_config.environment.docker.base_image = "pytorch/pytorch"
amlcompute_run_config.environment.spark.precache_packages = False

In [ ]:
ffmpeg_audio = PipelineData(name="ffmpeg_audio", datastore=default_datastore)
ffmpeg_images = PipelineData(name="ffmpeg_images", datastore=default_datastore)
processed_images = PipelineData(name="processed_images", datastore=default_datastore)
output_video = PipelineData(name="output_video", datastore=default_datastore)

# Define tweakable parameters to pipeline
These parameters can be changed when the pipeline is published and rerun from a REST call

In [ ]:
from azureml.pipeline.core.graph import PipelineParameter
# create a parameter for style (one of "candy", "mosaic", "rain_princess", "udnie") to transfer the images to
style_param = PipelineParameter(name="style", default_value="mosaic")
# create a parameter for the number of nodes to use in step no. 2 (style transfer)
nodecount_param = PipelineParameter(name="nodecount", default_value=1)

In [ ]:
split_video_step = PythonScriptStep(
    name="split video",
    script_name="process_video.py",
    arguments=["--input_video", orangutan_video,
               "--output_audio", ffmpeg_audio,
               "--output_images", ffmpeg_images,
              ],
    compute_target=cpu_cluster,
    inputs=[orangutan_video],
    outputs=[ffmpeg_images, ffmpeg_audio],
    runconfig=amlcompute_run_config,
    source_directory=scripts_folder
)

# create a MPI step for distributing style transfer step across multiple nodes in AmlCompute 
# using 'nodecount_param' PipelineParameter
distributed_style_transfer_step = MpiStep(
    name="mpi style transfer",
    script_name="neural_style_mpi.py",
    arguments=["--content-dir", ffmpeg_images,
               "--output-dir", processed_images,
               "--model-dir", models_dir,
               "--style", style_param,
               "--cuda", 1
              ],
    compute_target=gpu_cluster,
    node_count=nodecount_param, 
    process_count_per_node=1,
    inputs=[models_dir, ffmpeg_images],
    outputs=[processed_images],
    pip_packages=["mpi4py", "torch", "torchvision"],
    runconfig=amlcompute_run_config,
    use_gpu=True,
    source_directory=scripts_folder
)

stitch_video_step = PythonScriptStep(
    name="stitch",
    script_name="stitch_video.py",
    arguments=["--images_dir", processed_images, 
               "--input_audio", ffmpeg_audio, 
               "--output_dir", output_video],
    compute_target=cpu_cluster,
    inputs=[processed_images, ffmpeg_audio],
    outputs=[output_video],
    runconfig=amlcompute_run_config,
    source_directory=scripts_folder
)

# Run the pipeline

In [ ]:
pipeline = Pipeline(workspace=ws, steps=[stitch_video_step])
# submit the pipeline and provide values for the PipelineParameters used in the pipeline
pipeline_run = Experiment(ws, 'style_transfer').submit(pipeline, pipeline_params={"style": "mosaic", "nodecount": 3})

# Monitor using widget

In [ ]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

Downloads the video in `output_video` folder

# Download output video

In [ ]:
def download_video(run, target_dir=None):
    stitch_run = run.find_step_run("stitch")[0]
    port_data = stitch_run.get_output_data("output_video")
    port_data.download(target_dir, show_progress=True)

In [ ]:
pipeline_run.wait_for_completion()
download_video(pipeline_run, "output_video_mosaic")

# Publish pipeline

In [ ]:
published_pipeline = pipeline_run.publish_pipeline(
    name="batch score style transfer", description="style transfer", version="1.0")

published_id = published_pipeline.id

#  Re-run pipeline through REST calls for other styles

## Get AAD token

In [ ]:
from azureml.core.authentication import AzureCliAuthentication
import requests

cli_auth = AzureCliAuthentication()
aad_token = cli_auth.get_authentication_header()

## Get endpoint URL

In [ ]:
rest_endpoint = published_pipeline.endpoint

## Send request and monitor

In [ ]:
# run the pipeline using PipelineParameter values style='candy' and nodecount=2
response = requests.post(rest_endpoint, 
                         headers=aad_token,
                         json={"ExperimentName": "style_transfer",
                               "ParameterAssignments": {"style": "candy", "nodecount": 2}})                         
run_id = response.json()["Id"]

from azureml.pipeline.core.run import PipelineRun
published_pipeline_run_candy = PipelineRun(ws.experiments["style_transfer"], run_id)

RunDetails(published_pipeline_run_candy).show()

In [ ]:
# run the pipeline using PipelineParameter values style='rain_princess' and nodecount=3
response = requests.post(rest_endpoint, 
                         headers=aad_token,
                         json={"ExperimentName": "style_transfer",
                               "ParameterAssignments": {"style": "rain_princess", "nodecount": 3}})    
run_id = response.json()["Id"]

from azureml.pipeline.core.run import PipelineRun
published_pipeline_run_rain = PipelineRun(ws.experiments["style_transfer"], run_id)

RunDetails(published_pipeline_run_rain).show()

In [ ]:
# run the pipeline using PipelineParameter values style='udnie' and nodecount=4
response = requests.post(rest_endpoint, 
                         headers=aad_token,
                         json={"ExperimentName": "style_transfer",
                               "ParameterAssignments": {"style": "udnie", "nodecount": 4}})   
run_id = response.json()["Id"]

from azureml.pipeline.core.run import PipelineRun
published_pipeline_run_udnie = PipelineRun(ws.experiments["style_transfer"], run_id)

RunDetails(published_pipeline_run_udnie).show()

## Download output from re-run

In [ ]:
published_pipeline_run_candy.wait_for_completion()
published_pipeline_run_rain.wait_for_completion()
published_pipeline_run_udnie.wait_for_completion()

In [ ]:
download_video(published_pipeline_run_candy, target_dir="output_video_candy")
download_video(published_pipeline_run_rain, target_dir="output_video_rain_princess")
download_video(published_pipeline_run_udnie, target_dir="output_video_udnie")